In [0]:
import numpy as np
import sys

#X[TIEMPO,VARIABLE]



def selvar_python(X,maxlag,bs,itmax):
  (t_max,d)=X.shape
  lags=np.zeros((d,d))
  scores=np.zeros((d,d))

  for i in range(d):
    response=X[maxlag+1:t_max,:]
    prss=getscore(X,lags,i,maxlag,bs,t_max,d,'PRSS')

    iteracion=0
    while True:
      iteracion+=1
      if iteracion>=itmax:
        break
      (sel,lagsel,nuevo_prss)=mejor_actualizacion(X,i,t_max,d,prss,lags)
      if sel>0:
        lags[i,sel]=lagsel
      if sel==0:
        break
    scores[i,:]=puntuaciones(X,lags,i,maxlag,bs,t_max,d)
  return scores



def mejor_actualizacion(X,x_i,t_max,d,prss,lags):
  '''
  Input:
    X datos
    x_i variable a estimar
    t_max tiempo maximo
    d numero de variables
    press puntuacion mejor modelo
    lags lags del modelo 
  Output:
    sel: variable seleccionada
    l:lag seleccionado
    nuevo_prss: mejor puntuacion
  '''
  sel=0
  for j in range(d):
    for l in range(1,maxlag):
      if j==i or lags[i,j]==l:
        continue
      tmp=lags[i,j]
      lags[i,j]=l
      nuevo_prss=getscore(X,lags,i,maxlag,bs,t_max,d,'PRSS')  
      if nuevo_prss<prss:
        (sel,lagsel,prss)=(j,l,nuevo_prss)
      lags[i,j]=tmp
    return sel,lagsel,nuevo_prss


def getscore(X , lags, x_i, maxlag, bs,t_max,d, tipo, variable='nada'):
  '''
  Input:
datos = valores de las series temporales
lags  = lags del modelo para la variable x_i
x_i   = variable a predecir
maxlag= valor maximo del lag
tipo= RSS  o  PRSS
variable= si es necesario quitar alguna variable del modelo.
  Output:
prss= predicted RSS  para la variable.
  '''
  if (maxlag>t_max or maxlag<1): maxlag=1
  if (bs<0): bs=(t_max-maxlab)/(-bs)
  if (bs>(t_max-maxlag)): bs=t_max-maxlag
  if variable!='nada':
    lags[x_i,variable]=0

  score_modelo=0
  epochs=int((t_max-maxlag)/bs)
  lags=lags[x_i,:]


  for epoch in range(epochs):

    inicio=maxlag+epoch*bs
    final =maxlag+(epoch+1)*bs

    X_i=X[inicio :final,x_i]

    X_modelo=X[:,lags>0]
    lags=lags[lags>0]
    X_js=np.zeros((bs,X_modelo.shape[0]))

    for x_j in range(X_modelo.shape[1]):
      X_js[:,x_j]=X[inicio-lags[x_i,x_j]:final-lags[x_i,x_j],x_j]

    score_modelo = score_modelo+score(X_js,X_i,tipo)
  return score




def score(X,Y,tipo):
  '''
  Input:
  X atributos
  Y clases
  tipo RSS o PRSS


  Y=X*b+a
  H=X*inv(X.T*X)*X.T
  Y_pred=H*Y

  PRESS=sum((e_i/(1-h_i))^2)

  e_i=y-y_pred
  h_i=autovalor h_i
  '''
  N=Y.shape[0]
  try:
    H=np.dot(X,np.dot(np.linalg.inv(np.dot(X.T,X)),X.T))
  except:
    X=X+0.00001*np.random.rand(X.shape[0],X.shape[1])
    H=np.dot(X,np.dot(np.linalg.inv(np.dot(X.T,X)),X.T))
  e=Y-np.dot(H,Y)

  if tipo=='RSS':
    return np.dot(e,np.conj(e))
  if tipo=='PRSS':
    h=1-np.linalg.eigvals(H)
    E=e/h 
    PRSS=np.dot(E,np.conj(E))/N
  return PRSS

def puntuaciones(X,lags,i,maxlag,bs,t_max,d):
  scores=np.zeros(d)
  score=getscore(X,lags,i,maxlag,bs,t_max,d,'RSS')
  for j in range(d):
    if j==i:
      continue
    if lags[i,j]>0:
      scores[j]=getscore(X,lags,i,maxlag,bs,t_max,d,'RSS',j)-score
  return scores
  

In [0]:
X=np.random.rand(10000,10)


In [0]:
selvar_python(X,10,100,100)

In [54]:
a=np.random.rand(10,10)
a=np.identity(10)
print(a)
np.linalg.eigvals(a)



[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [78]:
x=np.random.rand(10,10)
h=1-np.linalg.eigvals(x)
y=np.random.rand(10)

a=y/h

print(a)


[-0.25560927-0.j          0.13106861+0.00795868j  0.18464655-0.01121202j
  0.71154765+0.71406015j  0.5152808 -0.51710028j  0.07733669+0.01417018j
  0.5136494 -0.09411447j  0.71925557+0.j          0.59292963+0.j
  0.21642907+0.j        ]
